In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from io import StringIO
from sklearn.preprocessing import MaxAbsScaler
import matplotlib.pyplot as plt
from IPython.display import clear_output
from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Electrics
import keyboard
import time

def millis():
    return time.time_ns() / 1000000
oldtime = millis()
curtime = 0
rectime = 0
step = 1/60 * 1000 #60 hz

## Lectura de datos

In [3]:
# Leer carpeta
CARPETA = "german"
datos = False
columnas = [
    # Comandos
    "throttle_input",
    "brake_input",
    "clutch_input",
    "steering_input",
    "gear",
    # Estado
    "rpm",
    "airspeed"
]
files = Path(CARPETA).glob('*')
for i, ruta in enumerate(files):
    df = pd.read_csv(ruta, usecols=columnas)
    if (i == 0):
        datos = df
    else:
        datos = pd.concat([datos, df])
datos = datos.sort_index(axis=1)

In [4]:
#aceleración
accel = [0]
datos_np = datos.to_numpy()
for i in range(datos.shape[0]):
    if i == 0:
        continue
    accel.append(datos_np[i, 0] - datos_np[i-1, 0])
datos["accel"] = accel
datos.drop(datos[datos["accel"]<-0.2].index, inplace=True)

## Interpretación

In [5]:
def interp(x):
    inp = 0
    throttle = x["throttle_input"].iloc[0]
    brake = x["brake_input"].iloc[0]
    clutch = x["clutch_input"].iloc[0]
    speed = x["airspeed"].iloc[0]
    accel = x["accel"].iloc[0]
    rpm = x["rpm"].iloc[0]
    if accel > 0.01:
        inp = 1
    elif (accel >= -0.01 and accel <= 0.01):
        inp = 0
        if (clutch > 0 and throttle > 0):
            inp = 1
        elif clutch > 0:
            inp = -1
    elif accel < -0.01:
        inp = -1
        if throttle > 0:
            inp = 1
        elif brake > 0:
            if speed > 1:
                inp = -1
    return inp
inte = []
for i in range(datos.shape[0]):
    inte.append(interp(datos[i:i+1]))
datos["interp"] = inte
display(datos)

,airspeed,brake_input,clutch_input,gear,rpm,steering_input,throttle_input,accel,interp
1,0.000133,0.0,0.0,0.0,667.582291,0.000000,0.0,-0.000018,0
2,0.000143,0.0,0.0,0.0,664.387431,0.000000,0.0,0.000010,0
3,0.000144,0.0,0.0,0.0,661.140019,0.000000,0.0,0.000002,0
4,0.000142,0.0,0.0,0.0,664.203268,0.000000,0.0,-0.000002,0
5,0.000161,0.0,0.0,0.0,667.470202,0.000000,0.0,0.000018,0
...,...,...,...,...,...,...,...,...,...
19105,19.755055,0.0,0.0,3.0,2750.780320,0.392376,0.0,-0.122945,-1
19106,19.628862,0.0,0.0,3.0,2688.020298,0.291094,0.0,-0.126193,-1
19107,19.498466,0.0,0.0,3.0,2617.569343,0.199892,0.0,-0.130396,-1
19108,19.364819,0.0,0.0,3.0,2541.855740,0.115181,0.0,-0.133648,-1


## ANN

## Verificación

In [4]:
bng_path = "C:\Program Files (x86)\Steam\steamapps\common\BeamNG.drive"
bng = BeamNGpy('localhost', 64256, home=bng_path)
bng.open()

In [5]:
scenario = Scenario('west_coast_usa', 'ml_beamng')
vehicle = Vehicle('veh', model='pessima', licence='HOLA')
electrics = Electrics()
vehicle.attach_sensor("electrics", electrics)
scenario.add_vehicle(vehicle, pos=(-717, 101, 118), rot_quat=(0, 0, 0.3826834, 0.9238795))
scenario.make(bng)
bng.load_scenario(scenario)
bng.start_scenario()
vehicle.set_shift_mode("realistic_manual")

In [70]:
def lectura_sensores():
    vehicle.poll_sensors()
    keys = ""
    val = ""
    for i, data in enumerate(electrics.data.items()):
        if (i > 0):
            keys = keys + ","
            val = val + ","
        if (data[0] == "wheelThermals"):
            continue
        keys = keys + str(data[0])
        val = val + str(data[1])
    return keys + '\n' + val

oldspeed = 0
while True:
    if (keyboard.is_pressed('D')):
        break
    #actualizar valor
    curtime = millis()
    if curtime > (oldtime + step):
        oldtime = curtime
        x = pd.read_csv(StringIO(lectura_sensores()), usecols=columnas).sort_index(axis=1)
        curspeed = x["airspeed"].iloc[0]
        accel = curspeed - oldspeed
        oldspeed = curspeed
        x["accel"] = accel
        display([interp(x), x["rpm"].iloc[0]])
        clear_output(wait = True)
        

[0, 671.2921083231324]

,airspeed,brake_input,clutch_input,gear,rpm,steering_input,throttle_input,accel
2,0.000143,0.0,0.0,0.0,664.387431,0.0,0.0,0.00001
